In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.models import resnet50
import time
import wandb
#
from leaf._core import LeafConfig, LeafTrainer

Initializing _core module...
_core module initialization complete!


In [2]:
config = LeafConfig()

In [ ]:
config.add_server(
    server_name="gpu-server-1",
    username="root",
    hostname="174.93.255.152",
    port=40056,  
)

In [4]:
# config.add_server(
#     server_name="gpu-server-1",
#     username="root",
#     hostname="120.238.149.138",
#     port=39363,  
# )

In [5]:
config.print_all_resources()


=== Available Servers and Resources ===

Server: gpu-server-1
Status: Not connected
Type: Remote server
Username: root
Hostname: 184.145.155.117
Port: 26765

No computational resources found

--------------------------------------------------

Server: localhost
Status: Connected
Type: Local machine

Available Resources (1):
--------------------------------------------------

Resource 1:
  Name: Apple M1
  Type: CPU
  Properties:

--------------------------------------------------

=== End of Server List ===



In [6]:
leaf_trainer = LeafTrainer(config)

In [7]:
leaf_trainer.test_with_hardcoded_values()

=== Testing get_gradients_from_server with hardcoded values ===

Testing server: gpu-server-1
Debug: Server gpu-server-1 has hostname: 184.145.155.117 and port: 26765
Server type: Remote
Connection status: Not connected
Skipping server gpu-server-1 - not connected

Testing server: localhost
Debug: Server localhost has hostname:  and port: 0
Server type: Local
Connection status: Connected
  Computing gradients locally (using GetGradients from server_communication)...
  Local computation completed
✓ Gradient computation successful!
  Loss: 0.5
  Gradients size: 3 elements
  Sample gradients: 4.59312e+27, 4.58281e+30, 6.88852e+22

Gradient testing with hardcoded values completed!


{'server_results': [{'server_name': 'gpu-server-1',
   'is_local': False,
   'is_connected': False,
   'success': False,
   'error': 'Server not connected'},
  {'server_name': 'localhost',
   'is_local': True,
   'is_connected': True,
   'success': True,
   'loss': 0.5,
   'gradients_size': 3,
   'gradients': [4.593116492825124e+27,
    4.582813224188066e+30,
    6.888519090641375e+22]}],
 'total_servers': 2}

In [8]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
num_epochs = 50
batch_size = 128
learning_rate = 0.001

# Data preprocessing
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [9]:
# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                      download=True, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=batch_size,
                        shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                     download=True, transform=transform_test)
testloader = DataLoader(testset, batch_size=batch_size,
                       shuffle=False, num_workers=2)

In [10]:
# Load pretrained ResNet-50 and modify for CIFAR-10
model = resnet50(pretrained=True)
model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
model.maxpool = nn.Identity()  # Remove maxpool as CIFAR-10 images are small
model.fc = nn.Linear(model.fc.in_features, 10)  # Change output to 10 classes
model = model.to(device)

/Users/ryanmarr/Documents/Subnautica_mod/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/ryanmarr/Documents/Subnautica_mod/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
# run = wandb.init(
#     project="big-model-example"
# )

In [12]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

In [13]:
for idx, (input, target) in enumerate(trainloader):
    print(idx)
    if idx == 0:
        i = input
        break

0


In [14]:
i

tensor([[[[-1.2078, -1.1497, -1.0721,  ..., -2.4291, -2.4291, -2.4291],
          [-1.2660, -0.9946, -0.6263,  ..., -2.4291, -2.4291, -2.4291],
          [-1.4211, -1.2466, -0.7813,  ..., -2.4291, -2.4291, -2.4291],
          ...,
          [-2.4291, -2.4291, -2.4291,  ..., -2.4291, -2.4291, -2.4291],
          [-2.4291, -2.4291, -2.4291,  ..., -2.4291, -2.4291, -2.4291],
          [-2.4291, -2.4291, -2.4291,  ..., -2.4291, -2.4291, -2.4291]],

         [[-1.3956, -1.3366, -1.2382,  ..., -2.4183, -2.4183, -2.4183],
          [-1.4742, -1.2579, -0.9629,  ..., -2.4183, -2.4183, -2.4183],
          [-1.6316, -1.4546, -1.0809,  ..., -2.4183, -2.4183, -2.4183],
          ...,
          [-2.4183, -2.4183, -2.4183,  ..., -2.4183, -2.4183, -2.4183],
          [-2.4183, -2.4183, -2.4183,  ..., -2.4183, -2.4183, -2.4183],
          [-2.4183, -2.4183, -2.4183,  ..., -2.4183, -2.4183, -2.4183]],

         [[-1.7336, -1.6166, -1.4995,  ..., -2.2214, -2.2214, -2.2214],
          [-1.7727, -1.6556, -

In [15]:
model = leaf_trainer.register_model(model)
#input, target = trainloader[0]
# criterion = leaf_trainer.register_criterion(criterion)
# optimizer = leaf_trainer.register_optimizer(optimizer)

# outputs = model(input)
# loss = criterion(outputs=outputs, target=target)
# loss.backward()
# optimizer.step()

Registering model with LeafTrainer...
Model state extracted, size: 23574015 parameters
Distributing model to 2 servers...
Debug: Server gpu-server-1 has hostname: 184.145.155.117 and port: 26765
Skipping server gpu-server-1 - not connected
Debug: Server localhost has hostname:  and port: 0
Storing model on server: localhost
✓ Model stored locally successfully
Model registered successfully! Total models: 1


In [16]:
asdf

NameError: name 'asdf' is not defined

In [12]:
# Training loop
def train():
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        if (batch_idx + 1) % 100 == 0:
            print(f'Batch: {batch_idx + 1} | Loss: {running_loss/(batch_idx + 1):.3f} | '
                  f'Acc: {100.*correct/total:.2f}%')
    
    return running_loss/len(trainloader), 100.*correct/total

In [13]:
# Testing loop
def test():
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, targets in testloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    
    return test_loss/len(testloader), 100.*correct/total

In [ ]:
# Main training loop
print('Starting training...')
best_acc = 0
for epoch in range(num_epochs):
    start_time = time.time()
    
    train_loss, train_acc = train()
    test_loss, test_acc = test()
    
    scheduler.step(test_loss)
    
    print(f'\nEpoch: {epoch + 1}/{num_epochs}')
    print(f'Time: {time.time() - start_time:.2f}s')
    print(f'Train Loss: {train_loss:.3f} | Train Acc: {train_acc:.2f}%')
    print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc:.2f}%')
    
    # Save best model
    if test_acc > best_acc:
        print('Saving best model...')
        state = {
            'model': model.state_dict(),
            'acc': test_acc,
            'epoch': epoch,
        }
        torch.save(state, 'best_model.pth')
        best_acc = test_acc
print('Training completed!')